# Imports e inicialización del servidor

In [1]:
import mysql.connector
import os, getpass, re, requests
import gensim, unidecode
import wikipedia
import pandas as pd
import numpy as np
import cvxpy as cp
import tensorflow as tf
from tqdm import tqdm
from mysql.connector import Error
from itertools import combinations
from ast import literal_eval
from collections import defaultdict
from scipy.spatial import distance
from bs4 import BeautifulSoup
from urllib.request import urlopen
K = 10

In [2]:
# Definición de parámetros de conexión
connection_params = {
    'host': 'localhost',
    'user': 'cmescobar',
    'database': 'foodb',
    'password': getpass.getpass(prompt='Introduzca la contraseña: ')
}

try:
    connection = mysql.connector.connect(**connection_params)

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()

except Error as e:
    print("Error while connecting to MySQL", e)

Introduzca la contraseña: ········
Connected to MySQL Server version  8.0.26


# Funciones de scrapping

In [3]:
def bs4_scraping_OLD(page):
    # Especificando la dirección URL
    source = urlopen(f'https://en.wikipedia.org/wiki/{page}').read()
    
    # Preparando la sopa 
    soup = BeautifulSoup(source,'lxml')
    
    # Extrayendo el texto de los párrafos
    text_out = ''
    for paragraph in soup.find_all('p'):
        text_out += paragraph.text

    return text_out


def bs4_scraping(page):
    # Acondicionando el link de la página
    page = page.replace(' ', '_')
    page = unidecode.unidecode(page)
    
    # Especificando la dirección URL
    source = urlopen(f'https://en.wikipedia.org/wiki/{page}').read()

    # Preparando la sopa 
    soup = BeautifulSoup(source,'lxml')

    # Extrayendo el texto de los párrafos
    paragraphs = []
    for paragraph in soup.find_all('p'):
        paragraphs.append(str(paragraph.text))
    
    # Uniendo como texto plano
    text_out = ' '.join(paragraphs)
    
    # Dropear footnotes en brackets
    text_out = re.sub(r"\[.*?\]+", '', text_out)
    
    # Eliminar los non-breaking-spaces
    text_out = re.sub(r"\xa0", ' ', text_out)
    
    # Eliminar los code promotion
    text_out = re.sub(r"code: lat promoted to code: la ", '', text_out)

    # Replace '\n' (a new line) with '' and end the string at $1000.
    text_out = text_out.replace('\n', '')
    
    # Remover espacios extra
    text_out = re.sub(r'\s+', ' ', text_out)
    
    return text_out


def wiki_scraping(page):
    # Obteniendo el texto de la página de wikipedia
    text_out = wikipedia.WikipediaPage(title=page).content
    print(text_out)
    # Limpiando el texto
    text_out = re.sub(r'==.*?==+', '', text_out)
    text_out = text_out.replace('\n', '')
    
    return text_out

# Obtener páginas desde wikipedia 

In [4]:
query = \
''' SELECT f.id, f.name, f.name_scientific, f.description, f.wikipedia_id
    FROM foods f
    WHERE f.wikipedia_id IS NOT NULL
    AND f.wikipedia_id != '';
'''

# Solicitud de la query
df = pd.read_sql(query, connection)
df

,id,name,name_scientific,description,wikipedia_id
0,1,Angelica,Angelica keiskei,Angelica is a genus of about 60 species of tal...,Angelica
1,2,Savoy cabbage,Brassica oleracea var. sabauda,Savoy cabbage (Brassica oleracea convar. capit...,Savoy cabbage
2,3,Silver linden,Tilia argentea,Tilia tomentosa (Silver Lime in the UK and Sil...,Tilia tomentosa
3,4,Kiwi,Actinidia chinensis,"The kiwifruit, often shortened to kiwi in many...",Kiwifruit
4,5,Allium,Allium,Allium haematochiton is a species of wild onio...,Allium haematochiton
...,...,...,...,...,...
848,934,Asparagus fern,Asparagus setaceus,"Asparagus setaceus, commonly called common asp...",Asparagus_setaceus
849,935,Thornless blackberry,Rubus ulmifolius,"Rubus ulmifolius, commonly called thornless bl...",Rubus_ulmifolius
850,936,Tropical highland blackberry,Rubus adenotrichos,Rubus adenotrichos is commonly called tropical...,Rubus_adenotrichos
851,937,Andean blackberry,Rubus glaucus,"Rubus glaucus, commonly called Andean blackber...",Rubus_glaucus


In [5]:
X_desc = df['description']
X_wiki = df['wikipedia_id']

In [6]:
X_wiki[0]

'Angelica'

# Beautiful Soup

In [7]:
bs4_scraping(X_wiki[5])

' The onion (Allium cepa L., from Latin cepa "onion"), also known as the bulb onion or common onion, is a vegetable that is the most widely cultivated species of the genus Allium. The shallot is a botanical variety of the onion. Until 2010, the shallot was classified as a separate species.: 21 Its close relatives include the garlic, scallion, leek, chive, and Chinese onion. This genus also contains several other species variously referred to as onions and cultivated for food, such as the Japanese bunching onion (Allium fistulosum), the tree onion (A. ×proliferum), and the Canada onion (Allium canadense). The name "wild onion" is applied to a number of Allium species, but A. cepa is exclusively known from cultivation. Its ancestral wild original form is not known, although escapes from cultivation have become established in some regions. The onion is most frequently a biennial or a perennial plant, but is usually treated as an annual and harvested in its first growing season. The onion 

# Wikipedia

In [8]:
_ = wiki_scraping(X_wiki[0])
print(_)

Angelica is a genus of about 60 species of tall biennial and perennial herbs in the family Apiaceae, native to temperate and subarctic regions of the Northern Hemisphere, reaching as far north as Iceland, Lapland and Greenland. They grow to 1–3 m (3 ft 3 in–9 ft 10 in) tall, with large bipinnate leaves and large compound umbels of white or greenish-white flowers. Found mainly in China, its main use was for medicine. It shows variations in fruit anatomy, leaf morphology and subterranean structures. The genes are extremely polymorphic.
Some species can be found in purple moor and rush pastures.


== Characteristics ==
Angelica species grow to 1–3 m (3 ft 3 in–9 ft 10 in) tall, with large bipinnate leaves and large compound umbels of white or greenish-white flowers. Their large, sparkling, starburst flowers are pollinated by a great variety of insects (the generalist pollination syndrome), the floral scents are species-specific, and even specific to particular subspecies. The active ingre

# Aplicando web scraping con BS4 para obtener el texto de las páginas

In [9]:
# Definición de la columna de texto a añadir
text_wiki = list()

for page in tqdm(X_wiki, ncols=70, desc='Ingredient'):
    # Definición del texto que se tiene para cada columna
    try:
        text_to = bs4_scraping(page)
        text_wiki.append(text_to)
    except:
        print("Page not found:", page)
        text_wiki.append(None)

# Añadiendo al dataframe
df['wikipedia_text'] = text_wiki

Ingredient: 100%|███████████████████| 853/853 [13:34<00:00,  1.05it/s]


In [10]:
# Guardando
df.to_pickle('Summary/Food_wikitext.pkl')